In [2]:
!pwd

/home/jovyan/work


In [4]:
!pip install -q -r requirements.txt

In [5]:
import ollama
import chromadb
from llama_index.core import SimpleDirectoryReader

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /opt/conda/lib/python3.11/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [6]:
MODEL = "phi:latest"

documents = SimpleDirectoryReader("/home/jovyan/work/data/tmp").load_data()

In [7]:
client = chromadb.HttpClient(host="chromadb-vecdb", port=8000)

In [8]:
dir(client)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_add',
 '_admin_client',
 '_count',
 '_create_system_if_not_exists',
 '_delete',
 '_get',
 '_get_identifier_from_settings',
 '_identifier',
 '_identifier_to_system',
 '_modify',
 '_peek',
 '_populate_data_from_system',
 '_query',
 '_server',
 '_submit_client_start_event',
 '_system',
 '_update',
 '_upsert',
 '_validate_tenant_database',
 'clear_system_cache',
 'count_collections',
 'create_collection',
 'database',
 'delete_collection',
 'from_system',
 'get_collection',
 'get_max_batch_size',
 'get_or_create_collection',
 'get_setti

In [7]:
client.list_collections()[0].name

'docs'

In [8]:
# client.delete_collection??

In [9]:
collection = client.get_or_create_collection(name="docs")

In [26]:
# collection.delete?

In [56]:
# client.delete_collection(name='docs')

In [9]:
ollama_client = ollama.Client(host='http://ollama:11434')

In [10]:
ollama_chat_resp = ollama_client.chat(
    messages=[
        {
            'role': 'user',
            'content': f"Explain the concept of black hole!",
        }
    ],
    model='phi:latest',
    stream=False
)

In [11]:
ollama_chat_resp['message']['content']

" I'd be happy to help you understand the concept of black holes. \n\na black hole is a region in space where gravity is so strong that nothing, not even light, can escape from it. this includes stars, gas, and other objects that get too close to its event horizon - the boundary beyond which nothing can escape. \n\nwhen an object falls into a black hole's event horizon, it's pulled by the gravitational force towards the center of the black hole. as more matter is added, the black hole continues to grow in size, until it becomes extremely dense and has an incredibly strong gravitational pull. \n\nthis intense gravity causes time to slow down around the black hole, and creates a phenomenon known as time dilation. this means that time passes at different rates for objects closer to the black hole compared to those further away from it. \n\nthe concept of a black hole has been around since ancient times, but wasn't fully understood until the 20th century when scientists like hurwitz, berke

In [87]:
stream_resp = ollama_client.chat(
    model='phi:latest',
    messages=[{'role': 'user', 'content': 'Why is the sky blue?'}],
    stream=True,
)

In [83]:
t = ['ravi', 'choudhary']
t[-1] = 'kumar'
t

['ravi', 'kumar']

In [88]:
#test_txt = []
full_response = ""
for chunk in stream_resp:
    msg = chunk['message']['content']
    # full_response.append(msg)
    full_response += msg
    print(msg, end='', flush=True)
    #test_txt.append(chunk['message']['content'])
    #print(test_txt)
    #print(chunk['message']['content'], end='', flush=True)

 As an AI language model, I can tell you that the sky appears blue due to the phenomenon called Rayleigh scattering, which occurs when sunlight interacts with molecules in Earth's atmosphere. Specifically, blue light is scattered more than other colors because it has a shorter wavelength and higher frequency, causing it to be reflected back to our eyes from all directions. This creates the perception of a blue sky.


In [49]:
# del stream_resp

In [89]:
full_response

" As an AI language model, I can tell you that the sky appears blue due to the phenomenon called Rayleigh scattering, which occurs when sunlight interacts with molecules in Earth's atmosphere. Specifically, blue light is scattered more than other colors because it has a shorter wavelength and higher frequency, causing it to be reflected back to our eyes from all directions. This creates the perception of a blue sky.\n"

In [90]:
# ''.join(full_response)

In [11]:
documents[0].metadata

{'page_label': '1',
 'file_name': 'agnipath_defence_poilicy.pdf',
 'file_path': '/home/jovyan/work/data/tmp/agnipath_defence_poilicy.pdf',
 'file_type': 'application/pdf',
 'file_size': 131092,
 'creation_date': '2024-09-24',
 'last_modified_date': '2024-09-24'}

In [36]:
# dir(collection)

In [22]:
# store each document in a vector embedding database
for i, doc in enumerate(documents):
  # get the document id and text
  doc_id = doc.id_
  doc_text = doc.get_text()
  doc_metadata = doc.metadata
  response = ollama_client.embeddings(model=MODEL, prompt=doc_text)
  embedding = response["embedding"]
  collection.add(
    ids=[doc_id],
    embeddings=[embedding],
    documents=[doc_text],
    metadatas=doc_metadata
  )

In [59]:
# client.get_collection?

In [27]:
col_load = client.get_collection(name="docs")

In [61]:
prompt = "what is agnipath scheme? What are the benefits and for whom?"

In [62]:
# generate an embedding for the prompt and retrieve the most relevant doc
response = ollama_client.embeddings(
  prompt=prompt,
  model=MODEL
)

In [63]:
results = col_load.query(
  query_embeddings=[response["embedding"]],
  n_results=2
)

In [64]:
results

{'ids': [['8e0b53c5-f68d-45e4-92a6-f61f84e5d241',
   'e4005970-9a8f-4be2-b97c-ff6049cf3f16']],
 'distances': [[4737.008870330587, 5455.332635399611]],
 'embeddings': None,
 'metadatas': [[{'creation_date': '2024-09-24',
    'file_name': 'agnipath_defence_poilicy.pdf',
    'file_path': '/home/jovyan/work/data/tmp/agnipath_defence_poilicy.pdf',
    'file_size': 131092,
    'file_type': 'application/pdf',
    'last_modified_date': '2024-09-24',
    'page_label': '4'},
   {'creation_date': '2024-09-24',
    'file_name': 'agnipath_defence_poilicy.pdf',
    'file_path': '/home/jovyan/work/data/tmp/agnipath_defence_poilicy.pdf',
    'file_size': 131092,
    'file_type': 'application/pdf',
    'last_modified_date': '2024-09-24',
    'page_label': '2'}]],
 'documents': [[' \n be enrolled in regular cadre of the Armed Forces. Detailed guidelines will be \nissued separately.  \n \n Enrolment will be undertaken through an online centralised system for \nall three services with specialised rallies 

In [30]:
results['documents'][0]

[' \n be enrolled in regular cadre of the Armed Forces. Detailed guidelines will be \nissued separately.  \n \n Enrolment will be undertaken through an online centralised system for \nall three services with specialised rallies and  campus interviews from \nrecognised technical institutes such as Industrial Training Institutes and \nNational Skills Qualifications Framework,  among others.  Enrolment will be \nbased on ‘All India All Class’ basis and the eligible age will be in range from \n17.5 to 21 years. Agniveers  will meet the medical eligibility conditions laid \ndown for enrolment in the armed forces as applicable to respective \ncategories/trades. The educational qualification for Agniveers will remain as \nin vogue for enrollment in various categories. {For example: For entry into \nGeneral Duty (GD) soldier, the educational qualification is Class 10}.  \n \n \n \n \n ',
 ' \n scheme.  The national stands to immensely benefit by infusion of highly \ninspired youth with deeper 

In [31]:
data = " ".join(results['documents'][0])

In [33]:
# generate a response combining the prompt and data we retrieved in step 2
output = ollama_client.generate(
  model=MODEL,
  prompt=f"Using this data: {data}. Respond to this prompt: {prompt}"
)

print(output['response'])

 The Agnipath Scheme is a new recruitment policy implemented by the Government of India that aims to recruit young individuals into the three services, namely the Army, Navy and Air Force, through an online centralised system. Enrolment will be undertaken based on 'All-India All Class' basis and eligible candidates must meet the medical eligibility conditions laid down for enrolment in different categories or trades. The educational qualification for Agniveers remains unchanged for enrollment into various categories/trades such as General Duty soldier, where educational qualifications should not be less than Class 10. 
Agnipath Scheme offers a customised monthly package with Risk and Hardship allowances, which are to be paid upon completion of four years of service in the Armed Forces. Agniveers will also receive an attractive 'Seva Nidhi' package on completion of their engagement period that includes contribution towards corpus fund by the Government along with interest accrued thereo